In [2]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv(r'C:\Users\jakep\OneDrive\Documents\CS639\citibike-network\2019_citibike.csv',
                parse_dates = ['Start Time'],
                usecols = ['Trip Duration', 'Start Time', 'Start Station ID', 'End Station ID'])
print(df)

          Trip Duration              Start Time  Start Station ID  \
0                   320 2019-01-01 00:01:47.401              3160   
1                   316 2019-01-01 00:04:43.736               519   
2                   591 2019-01-01 00:06:03.997              3171   
3                  2719 2019-01-01 00:07:03.545               504   
4                   303 2019-01-01 00:07:35.945               229   
...                 ...                     ...               ...   
20551692             93 2019-12-31 23:58:18.016              3141   
20551693            786 2019-12-31 23:58:59.896               490   
20551694            351 2019-12-31 23:59:03.695              3349   
20551695           1571 2019-12-31 23:59:21.361               252   
20551696            310 2019-12-31 23:59:55.296              3351   

          End Station ID  
0                 3283.0  
1                  518.0  
2                 3154.0  
3                 3709.0  
4                  503.0  
...      

In [4]:
stations = df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).size()
stations = stations.reset_index()
stations

,start station id,start station name,start station latitude,start station longitude,0
0,72,W 52 St & 11 Ave,40.767272,-73.993929,4849
1,79,Franklin St & W Broadway,40.719116,-74.006667,2980
2,82,St James Pl & Pearl St,40.711174,-74.000165,1403
3,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,1962
4,116,W 17 St & 8 Ave,40.741776,-74.001497,6839
...,...,...,...,...,...
788,3787,Dock 72 Way & Market St,40.699850,-73.971410,516
789,3788,E 12 St & 4 Av,40.732690,-73.989740,3538
790,3790,Henry St & Middagh St,40.700300,-73.991581,1644
791,3793,Stuyvesant Walk & 1 Av Loop,40.732340,-73.978984,111


In [5]:
import shapefile
from shapely.geometry import shape, Point

r = shapefile.Reader(r"C:\Users\jakep\Downloads\Borough Boundaries\geo_export_43b35747-351a-4fbf-944b-eaee1f68385e.shp")
shapes = r.shapes()

polygon = shape(shapes[1])    

def check(lon, lat):
    point = Point(lon, lat)
    return polygon.contains(point)

stations['Manhattan'] = stations.apply(lambda x: check(x['start station longitude'], x['start station latitude']), axis=1)
stations

,start station id,start station name,start station latitude,start station longitude,0,Manhattan
0,72,W 52 St & 11 Ave,40.767272,-73.993929,4849,True
1,79,Franklin St & W Broadway,40.719116,-74.006667,2980,True
2,82,St James Pl & Pearl St,40.711174,-74.000165,1403,True
3,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,1962,False
4,116,W 17 St & 8 Ave,40.741776,-74.001497,6839,True
...,...,...,...,...,...,...
788,3787,Dock 72 Way & Market St,40.699850,-73.971410,516,False
789,3788,E 12 St & 4 Av,40.732690,-73.989740,3538,True
790,3790,Henry St & Middagh St,40.700300,-73.991581,1644,False
791,3793,Stuyvesant Walk & 1 Av Loop,40.732340,-73.978984,111,True


In [6]:
manhattan_stations = stations[stations['Manhattan']].copy()
manhattan_stations_ids = list(manhattan_stations['start station id'])
print(len(manhattan_stations_ids))
manhattan_stations

419


,start station id,start station name,start station latitude,start station longitude,0,Manhattan
0,72,W 52 St & 11 Ave,40.767272,-73.993929,4849,True
1,79,Franklin St & W Broadway,40.719116,-74.006667,2980,True
2,82,St James Pl & Pearl St,40.711174,-74.000165,1403,True
4,116,W 17 St & 8 Ave,40.741776,-74.001497,6839,True
7,127,Barrow St & Hudson St,40.731724,-74.006744,6544,True
...,...,...,...,...,...,...
758,3747,E 84 St & 3 Ave,40.777554,-73.955128,3242,True
759,3749,Lexington Ave & E 36 St,40.747574,-73.978801,2792,True
787,3783,Cliff St & Fulton St,40.708380,-74.004950,1908,True
789,3788,E 12 St & 4 Av,40.732690,-73.989740,3538,True


### Remove loops
We are not interested in rides that start and end at the same station.

In [7]:
same_station = df['start station id'] == df['end station id']
print(f'{sum(same_station)}')
df = df[~same_station]

53758


In [8]:
df.dtypes
df.loc[df['starttime'].dt.dayofweek < 5, 'Weekday'] = 'Weekday'
df.loc[df['starttime'].dt.dayofweek >= 5, 'Weekday'] = 'Weekend'
#df['Weekday'] = df['starttime'].dt.dayofweek < 5 else 'Weekend'
df.loc[(21 > df['starttime'].dt.hour) & (df['starttime'].dt.hour >= 7), 'Daytime'] = 'Day'
df.loc[(21 <= df['starttime'].dt.hour) | (df['starttime'].dt.hour < 7), 'Daytime'] = 'Night'
#df['Daytime'] = (9 > df['starttime'].dt.hour) & (df['starttime'].dt.hour >= 7)
#df['Daytime'] = (9 > df['starttime'].dt.hour) & (df['starttime'].dt.hour >= 7)
df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Weekday,Daytime
0,330,2019-06-01 00:00:01.500,2019-06-01 00:05:31.7600,3602,31 Ave & 34 St,40.763154,-73.920827,3570,35 Ave & 37 St,40.755733,-73.923661,20348,Subscriber,1992,1,Weekend,Night
1,830,2019-06-01 00:00:04.240,2019-06-01 00:13:55.1470,3054,Greene Ave & Throop Ave,40.689493,-73.942061,3781,Greene Av & Myrtle Av,40.698568,-73.918877,34007,Subscriber,1987,2,Weekend,Night
2,380,2019-06-01 00:00:06.019,2019-06-01 00:06:26.7790,229,Great Jones St,40.727434,-73.993790,326,E 11 St & 1 Ave,40.729538,-73.984267,20587,Subscriber,1990,2,Weekend,Night
3,1155,2019-06-01 00:00:06.776,2019-06-01 00:19:22.5380,3771,McKibbin St & Bogart St,40.706237,-73.933871,3016,Kent Ave & N 7 St,40.720368,-73.961651,33762,Subscriber,1987,1,Weekend,Night
4,1055,2019-06-01 00:00:07.520,2019-06-01 00:17:42.5580,441,E 52 St & 2 Ave,40.756014,-73.967416,3159,W 67 St & Broadway,40.774925,-73.982666,31290,Subscriber,1973,1,Weekend,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125365,232,2019-06-30 23:59:32.892,2019-07-01 00:03:25.6310,3113,Greenpoint Ave & Manhattan Ave,40.730260,-73.953940,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,25779,Subscriber,1972,1,Weekend,Night
2125366,477,2019-06-30 23:59:33.699,2019-07-01 00:07:31.0440,445,E 10 St & Avenue A,40.727408,-73.981420,3737,Stanton St & Norfolk St,40.720747,-73.986274,39522,Subscriber,1981,2,Weekend,Night
2125367,300,2019-06-30 23:59:47.405,2019-07-01 00:04:48.1850,3307,West End Ave & W 94 St,40.794165,-73.974124,3320,Central Park West & W 100 St,40.794067,-73.962868,26730,Subscriber,1981,1,Weekend,Night
2125368,906,2019-06-30 23:59:51.598,2019-07-01 00:14:58.3210,402,Broadway & E 22 St,40.740343,-73.989551,400,Pitt St & Stanton St,40.719261,-73.981780,38286,Subscriber,1996,1,Weekend,Night


In [11]:
# Given a point(lon, lat) determine whether is it located in Manhattan
def station_in_city(start, stop):
    s = start in manhattan_stations_ids
    e = stop in manhattan_stations_ids
    return s and e
    
df['Manhattan'] = df.apply(lambda x: station_in_city(x['start station id'], x['end station id']), axis = 1)

In [12]:
# Percentage of rides starting and ending in Manhattan
np.mean(df['Manhattan'])

0.744229131709992

In [13]:
# Only keep stations in Manhattan
df = df[df['Manhattan']]
df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Weekday,Daytime,Manhattan
2,380,2019-06-01 00:00:06.019,2019-06-01 00:06:26.7790,229,Great Jones St,40.727434,-73.993790,326,E 11 St & 1 Ave,40.729538,-73.984267,20587,Subscriber,1990,2,Weekend,Night,True
4,1055,2019-06-01 00:00:07.520,2019-06-01 00:17:42.5580,441,E 52 St & 2 Ave,40.756014,-73.967416,3159,W 67 St & Broadway,40.774925,-73.982666,31290,Subscriber,1973,1,Weekend,Night,True
5,128,2019-06-01 00:00:07.986,2019-06-01 00:02:16.3950,3236,W 42 St & Dyer Ave,40.758985,-73.993800,495,W 47 St & 10 Ave,40.762699,-73.993012,25137,Subscriber,1989,1,Weekend,Night,True
7,471,2019-06-01 00:00:14.644,2019-06-01 00:08:05.9930,3467,W Broadway & Spring Street,40.724947,-74.001659,401,Allen St & Rivington St,40.720196,-73.989978,26972,Subscriber,1990,1,Weekend,Night,True
8,1554,2019-06-01 00:00:18.197,2019-06-01 00:26:12.7890,379,W 31 St & 7 Ave,40.749156,-73.991600,2006,Central Park S & 6 Ave,40.765909,-73.976342,32969,Customer,1970,1,Weekend,Night,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125364,528,2019-06-30 23:59:32.080,2019-07-01 00:08:20.2200,514,12 Ave & W 40 St,40.760875,-74.002777,458,11 Ave & W 27 St,40.751396,-74.005226,15360,Customer,1969,0,Weekend,Night,True
2125366,477,2019-06-30 23:59:33.699,2019-07-01 00:07:31.0440,445,E 10 St & Avenue A,40.727408,-73.981420,3737,Stanton St & Norfolk St,40.720747,-73.986274,39522,Subscriber,1981,2,Weekend,Night,True
2125367,300,2019-06-30 23:59:47.405,2019-07-01 00:04:48.1850,3307,West End Ave & W 94 St,40.794165,-73.974124,3320,Central Park West & W 100 St,40.794067,-73.962868,26730,Subscriber,1981,1,Weekend,Night,True
2125368,906,2019-06-30 23:59:51.598,2019-07-01 00:14:58.3210,402,Broadway & E 22 St,40.740343,-73.989551,400,Pitt St & Stanton St,40.719261,-73.981780,38286,Subscriber,1996,1,Weekend,Night,True


In [16]:
summary = df.groupby(['start station name', 'end station name', 'Weekday', 'Daytime']).size()
summary = summary.unstack(fill_value=0).unstack(fill_value=0)
summary.columns = [' '.join(col).strip() for col in summary.columns.values]
summary

Day Weekday  Day Weekend  \
start station name    end station name                                     
1 Ave & E 110 St      1 Ave & E 44 St                     0            0   
                      1 Ave & E 62 St                     1            1   
                      1 Ave & E 68 St                     9            1   
                      1 Ave & E 78 St                     3            2   
                      1 Ave & E 94 St                    12            7   
...                                                     ...          ...   
Yankee Ferry Terminal South St & Gouverneur Ln            0            2   
                      South St & Whitehall St             1            5   
                      W 45 St & 8 Ave                     0            1   
                      West St & Chambers St               0            3   
                      West Thames St                      0            2   

                                                Night Weekday  Night Weekend  
start station name    end station name                                        
1 Ave & E 110 St      1 Ave & E 44 St                       1              0  
                      1 Ave & E 62 St                       0              0  
                      1 Ave & E 68 St                       0              0  
                      1 Ave & E 78 St                       2              0  
                      1 Ave & E 94 St                       0              0  
...                                                       ...            ...  
Yankee Ferry Terminal South St & Gouverneur Ln              0              0  
                      South St & Whitehall St               1              0  
                      W 45 St & 8 Ave                       0              0  
                      West St & Chambers St                 0              0  
                      West Thames St                        0              0  

[109674 rows x 4 columns]

In [17]:
summary['Day'] = summary['Day Weekday'] + summary['Day Weekend']
summary['Night'] = summary['Night Weekday'] + summary['Night Weekend']
summary['Weekday'] = summary['Night Weekday'] + summary['Day Weekday']
summary['Weekend'] = summary['Night Weekend'] + summary['Day Weekend']
summary['Total'] = summary['Day'] + summary['Night']
summary

Day Weekday  Day Weekend  \
start station name    end station name                                     
1 Ave & E 110 St      1 Ave & E 44 St                     0            0   
                      1 Ave & E 62 St                     1            1   
                      1 Ave & E 68 St                     9            1   
                      1 Ave & E 78 St                     3            2   
                      1 Ave & E 94 St                    12            7   
...                                                     ...          ...   
Yankee Ferry Terminal South St & Gouverneur Ln            0            2   
                      South St & Whitehall St             1            5   
                      W 45 St & 8 Ave                     0            1   
                      West St & Chambers St               0            3   
                      West Thames St                      0            2   

                                                Night Weekday  Night Weekend  \
start station name    end station name                                         
1 Ave & E 110 St      1 Ave & E 44 St                       1              0   
                      1 Ave & E 62 St                       0              0   
                      1 Ave & E 68 St                       0              0   
                      1 Ave & E 78 St                       2              0   
                      1 Ave & E 94 St                       0              0   
...                                                       ...            ...   
Yankee Ferry Terminal South St & Gouverneur Ln              0              0   
                      South St & Whitehall St               1              0   
                      W 45 St & 8 Ave                       0              0   
                      West St & Chambers St                 0              0   
                      West Thames St                        0              0   

                                                Day  Night  Weekday  Weekend  \
start station name    end station name                                         
1 Ave & E 110 St      1 Ave & E 44 St             0      1        1        0   
                      1 Ave & E 62 St             2      0        1        1   
                      1 Ave & E 68 St            10      0        9        1   
                      1 Ave & E 78 St             5      2        5        2   
                      1 Ave & E 94 St            19      0       12        7   
...                                             ...    ...      ...      ...   
Yankee Ferry Terminal South St & Gouverneur Ln    2      0        0        2   
                      South St & Whitehall St     6      1        2        5   
                      W 45 St & 8 Ave             1      0        0        1   
                      West St & Chambers St       3      0        0        3   
                      West Thames St              2      0        0        2   

                                                Total  
start station name    end station name                 
1 Ave & E 110 St      1 Ave & E 44 St               1  
                      1 Ave & E 62 St               2  
                      1 Ave & E 68 St              10  
                      1 Ave & E 78 St               7  
                      1 Ave & E 94 St              19  
...                                               ...  
Yankee Ferry Terminal South St & Gouverneur Ln      2  
                      South St & Whitehall St       7  
                      W 45 St & 8 Ave               1  
                      West St & Chambers St         3  
                      West Thames St                2  

[109674 rows x 9 columns]

In [18]:
summary = summary.reset_index()

In [19]:
for col in summary.select_dtypes(include=np.number).columns.tolist():
    summary.sort_values(col, inplace=True, ascending=False)
    summary[f'{col}_R'] = range(len(summary))

In [20]:
ranks = [col for col in summary if col.endswith('_R')]
ranks

['Day Weekday_R',
 'Day Weekend_R',
 'Night Weekday_R',
 'Night Weekend_R',
 'Day_R',
 'Night_R',
 'Weekday_R',
 'Weekend_R',
 'Total_R']

In [21]:
# Only keep rides where one of the combinations ranks in 1000 most frequent
summary = summary[(summary[ranks] < 1000).any(axis=1)]

In [25]:
stations = stations[stations.Manhattan]
stations

,start station id,start station name,start station latitude,start station longitude,0,Manhattan
0,72,W 52 St & 11 Ave,40.767272,-73.993929,4849,True
1,79,Franklin St & W Broadway,40.719116,-74.006667,2980,True
2,82,St James Pl & Pearl St,40.711174,-74.000165,1403,True
4,116,W 17 St & 8 Ave,40.741776,-74.001497,6839,True
7,127,Barrow St & Hudson St,40.731724,-74.006744,6544,True
...,...,...,...,...,...,...
758,3747,E 84 St & 3 Ave,40.777554,-73.955128,3242,True
759,3749,Lexington Ave & E 36 St,40.747574,-73.978801,2792,True
787,3783,Cliff St & Fulton St,40.708380,-74.004950,1908,True
789,3788,E 12 St & 4 Av,40.732690,-73.989740,3538,True


###  Export files to JSON

In [22]:
summary.to_json('paths.json', orient='index', indent=4)

In [26]:
stations.drop([0, 'Manhattan'], axis=1).set_index('start station id').to_json('stations.json', orient='records', indent=4)